## Задание SQL

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import time
from django.utils import dateparse
from datetime import timedelta
from datetime import datetime
import sqlite3
import random

In [2]:
try:
    sqlite_connection = sqlite3.connect('sqlite_python1.db')
    cursor = sqlite_connection.cursor()
    print("База данных создана и успешно подключена к SQLite")
    sqlite_select_query = "select sqlite_version();"
    cursor.execute(sqlite_select_query)
    record = cursor.fetchall()
    print("Версия базы данных SQLite: ", record)
    cursor.close()
except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)
finally:
    if (sqlite_connection):
        sqlite_connection.close()
        print("Соединение с SQLite закрыто")

База данных создана и успешно подключена к SQLite
Версия базы данных SQLite:  [('3.33.0',)]
Соединение с SQLite закрыто


In [3]:
def execute(sql):
    sqlite_connection = sqlite3.connect('sqlite_python1.db', timeout=10)
    cursor = sqlite_connection.cursor()
    cursor.execute(sql)
    sqlite_connection.commit()
    cursor.close()

def retexec(sql):
    sqlite_connection = sqlite3.connect('sqlite_python1.db', timeout=10)
    cursor = sqlite_connection.cursor()
    cursor.execute(sql)
    cur = cursor.fetchall()
    r = pd.DataFrame(cur)
    if cursor is not None and isinstance (cursor, sqlite3.Cursor):
        colnames = [row[0] for row in cursor.description]
        if len(r.columns) == len(colnames):
            r.columns = colnames
    sqlite_connection.commit()
    cursor.close()
    return r

def fill_table_many(table_name, dbs, datecolumns):
    start = datetime.now()
    data = dbs.copy()
    for c in data.columns:
        data[c] = data[c].apply(str)
    ve = [x.upper() for x in data.columns]
    vc = '", "'.join(ve)
    vc = '("'+vc+'")'
    vd = '", :"'.join(ve)
    vd = '(:"'+vd+'")'    
    ve = ['?' for x in data.columns]
    vc = ', '.join(ve)
    vc = '('+vc+')'
    sql = ("""insert into """+table_name+""" values """+vc)
    result = []
    for j in tqdm(range(len(data))):
        res = [data.loc[j, c] for c in data.columns]
        res = tuple(i for i in res)
        result.append(res)
    try:
        connection = sqlite3.connect('sqlite_python1.db', timeout=10)
        cur = connection.cursor()
        cur.executemany(sql,  result)
        connection.commit()
        cur.close()
    except:
        print(cur.rowcount)
    finish = datetime.now()
    t = '{}'.format(finish-start)
    print(t)

def create_table(table_name, ds, datecolumns):
    data = ds.copy()
    vc = []
    cat_col = [c for c in data.columns if data[c].dtype.name == 'object' if c not in datecolumns]
    num_col = [c for c in data.columns if data[c].dtype.name != 'object' if c not in datecolumns]
    for c in datecolumns:
        vc.append('"' +  c.upper() + ' TEXT"')
    for c in cat_col:
        vc.append('"' +  c.upper() + ' TEXT"')
    for c in num_col:
        vc.append('"' +  c.upper() + ' INTEGER"')
    vc = ', '.join(vc)
    sql = """create table """ +table_name+""" (  """ + vc +"""  )"""
    execute(sql)
    
def create_one_row(dt_start):
    v = ['"'+dt_start+'"']
    v.append('"'+str(dateparse.parse_datetime(dt_start).timestamp())+'"')
    v.append(int(random.choice(['1' for x in range(50)]+['2' for x in range(10)])))
    v.append(int(random.choice([str(x) for x in range(75)])))
    v.append(int(random.choice([str(x) for x in range(10,70)])))
    v.append('"'+random.choice(['Mobile','Desktop'])+'"')   
    v.append(int(random.choice([str(x) for x in range(10000)])))
    v.append(int(random.choice([str(x) for x in range(23)])))
    v.append(int(random.choice([str(x) for x in range(50000)])))
    v.append(int(random.choice([str(x) for x in range(90000)])))
    v.append(str(i))
    return v

## Создание таблицы events

### 1. Подготовка полей таблицы

In [4]:
dct = {'string':'TEXT', 'int':'INTEGER', 'date':'TEXT', 'datetime':'TEXT', 'boolean':'INTEGER'}

fields = 'date;timestamp;event_type;geo;age;device_type;Banner_id;Campaign_id;Amount;Pad_id;Event_id'
fields = fields.split(';')

types = 'date;datetime;int;int;int;string;int;int;int;int;int'
types = types.split(';')
types = [dct[x] for x in types]  
types[-1] = types[-1]+ ' PRIMARY KEY'
 
s = []
for i in range(len(fields)):
    s.append(' '.join([fields[i], types[i]]))
s = ', '.join(s)

### 2. Создание таблицы

In [5]:
table_name = 'events'
sql = """CREATE TABLE IF NOT EXISTS """+table_name+""" ( """+s+""");"""
execute(sql)

### 3. Заполнение таблицы синтезированными данными

In [6]:
dt = '01.02.2020 00:00:00'
dt = datetime.strptime(dt, "%d.%m.%Y %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")

days = 30
dbs = []
for i in tqdm(range(days)):
    dt_start = (dateparse.parse_datetime(dt)+timedelta(seconds=i*3600*24)).strftime("%Y-%m-%d %H:%M:%S")
    n = 0
    if dt_start == '2020-02-29 00:00:00':
        n = 50
    for j in range(30+n):
        v = create_one_row(dt_start)
        dbs.append(v)

dbs = pd.DataFrame(dbs, columns = fields)
dbs['Event_id'] = [str(x) for x in range(len(dbs))]

fill_table_many(table_name, dbs, datecolumns=[])

100%|██████████| 950/950 [00:00<00:00, 17215.10it/s]

0:00:00.066808


In [7]:
dbs = retexec("""select * from events""")

In [9]:
dbs.head(10)

,date,timestamp,event_type,geo,age,device_type,Banner_id,Campaign_id,Amount,Pad_id,Event_id
0,"""2020-02-01 00:00:00""","""1580504400.0""",1,7,46,"""Mobile""",7094,11,11233,60262,0
1,"""2020-02-01 00:00:00""","""1580504400.0""",1,26,54,"""Desktop""",9441,0,14674,76396,1
2,"""2020-02-01 00:00:00""","""1580504400.0""",1,59,16,"""Desktop""",7439,7,40777,61847,2
3,"""2020-02-01 00:00:00""","""1580504400.0""",1,7,14,"""Mobile""",8953,11,20504,20095,3
4,"""2020-02-01 00:00:00""","""1580504400.0""",1,27,30,"""Desktop""",6574,1,47211,43813,4
5,"""2020-02-01 00:00:00""","""1580504400.0""",1,31,31,"""Desktop""",8442,16,25909,65012,5
6,"""2020-02-01 00:00:00""","""1580504400.0""",1,17,65,"""Mobile""",2446,6,22275,47367,6
7,"""2020-02-01 00:00:00""","""1580504400.0""",2,20,46,"""Mobile""",3449,10,40104,89043,7
8,"""2020-02-01 00:00:00""","""1580504400.0""",1,22,12,"""Desktop""",8770,3,39135,69204,8
9,"""2020-02-01 00:00:00""","""1580504400.0""",1,18,62,"""Mobile""",2738,15,5061,82634,9


## Создание таблицы campaigns

In [10]:
fields = 'id;is_active;name;format'
fields = fields.split(';')

types = 'int;boolean;string;string'
types = types.split(';')
types = [dct[x] for x in types]
types[0] = types[0]+ ' PRIMARY KEY'

descr = 'Идентификатор рекламной кампании;Флаг активности кампании;Имя кампании;\
"Формат рекламы. Принимает значения: Video Carousel Multiv Image"'
descr = descr.split(';')

s = []
for i in range(len(fields)):
    s.append(' '.join([fields[i], types[i]]))
s = ', '.join(s)

sql = """CREATE TABLE IF NOT EXISTS campaigns ( """+s+""");"""
execute(sql)

In [11]:
for i in tqdm(range(25)):
    v = [i]
    v.append(int(random.choice(['0','1'])))
    v.append('"'+random.choice(['first','second','third','fourth'])+'"')    
    v.append('"'+random.choice(['Video','Carousel','Multiv','Image'])+'"')
    s2 = ','.join([str(x) for x in v])
    sql = """INSERT INTO campaigns ("""+', '.join(fields)+""")  VALUES  ("""+s2+""")"""
    execute(sql)

100%|██████████| 25/25 [00:00<00:00, 1026.91it/s]


In [13]:
dba = retexec("""select * from campaigns""")

In [14]:
dba.head(10)

,id,is_active,name,format
0,0,1,first,Multiv
1,1,0,fourth,Image
2,2,1,fourth,Carousel
3,3,1,fourth,Image
4,4,0,fourth,Carousel
5,5,0,second,Image
6,6,1,third,Video
7,7,1,third,Video
8,8,1,first,Multiv
9,9,1,first,Video


### Задание 1.

29.02.2020 по отношению к предыдущей неделе выросли показы мобильных пользователей. 

Необходимо проанализировать, изменилась ли структура показов по возрастным группам.

Напишите запрос, который подготовит данные для проведения такого анализа. 

Пользователей разбейте на 4 группы:
* до 18 лет
* с 18 до 60 лет
* старше 60 лет
* пользователи, чей возраст не известен

Структура итоговой таблицы:
* дата
* возрастная группа
* количество показов

In [15]:
db1 = retexec("""
   with e as
    (select *,
      case
        when (age < 18) then "<-18"
        when (age >= 18 and age < 60) then "18-60"
        when (age >= 60) then "60+"
        when (age is null) then "undefined"
      end age_d
    from events
    order by age)
        
    select date "дата", age_d "возрастная группа", count(*) "количество показов" from e  
    where event_type = 1 and device_type in ('"Mobile"')
    and date >= ('"2020-02-22 00:00:00"') and date <= ('"2020-02-29 00:00:00"')
    group by date, age_d, event_type  
    order by age_d, date  
        """)

In [16]:
db1

,дата,возрастная группа,количество показов
0,"""2020-02-22 00:00:00""",18-60,16
1,"""2020-02-23 00:00:00""",18-60,10
2,"""2020-02-24 00:00:00""",18-60,10
3,"""2020-02-25 00:00:00""",18-60,9
4,"""2020-02-26 00:00:00""",18-60,13
5,"""2020-02-27 00:00:00""",18-60,7
6,"""2020-02-28 00:00:00""",18-60,10
7,"""2020-02-29 00:00:00""",18-60,23
8,"""2020-02-22 00:00:00""",60+,3
9,"""2020-02-23 00:00:00""",60+,2


### Задание 2.

В феврале 2020 года по отношению к январю 2020 произошло снижение CTR 

(Click-Through Rate, отношение количества кликов (2) по объявлению к количеству его показов (1)). 

Вам нужно проанализировать причины снижения. Для этого:

Посчитайте CTR в разрезе формата рекламы за эти два периода. 

Структура итоговой таблицы:
* период (январь / февраль)
* формат рекламы
* CTR

In [25]:
db2 = retexec("""
 with c as (

    with b as (
        with a as      (
        select e.event_type, c.format, CAST(substr(date,7,2) AS INTEGER) month from events e
        left join (select id, format from campaigns) c on c.id = e.Campaign_id
                       )
        select month, format, event_type, count(event_type) cnt from a
        group by month, event_type, format
              )
    select
      s.month   as month,
      s.format  as format,
      CAST(s.cnt as REAL)      as show,
      CAST(e.cnt as REAL)      as click

    from (select month, event_type, format, cnt from b where event_type=1) as s
    left join (select month, event_type, format, cnt from b where event_type=2) as e
        on s.format = e.format and s.month = e.month
    group by s.month, s.cnt
    order by s.month, s.cnt
          )
    select month, format, (click / show) CTR from c
    order by month, CTR
             """)

In [26]:
db2

,month,format,CTR
0,2,Carousel,0.146341
1,2,Multiv,0.193182
2,2,Image,0.215470
3,2,Video,0.250000
4,3,Video,0.142857
5,3,Image,0.166667
6,3,Carousel,0.250000
7,3,Multiv,0.666667


### Задание 3.

Посчитайте CTR в разрезе площадок. 

В итоговую таблицу выведите топ-5 площадок, на которых был самый низкий CTR в феврале 2020 года. 

Структура итоговой таблицы:
* идентификатор площадки
* CTR за январь 2020 года
* CTR за февраль 2020 года

In [31]:
db3 = retexec("""

 with c as (

     with b as (
        with a as (select geo, CAST(substr(date,7,2) AS INTEGER) month, e.event_type from events e)
        select month, geo, event_type, count(event_type) cnt from a
        group by month, event_type, geo
               )
    select  
        s.month as month,
        s.geo as geo,
        CAST(s.cnt as REAL)  as show,
        CAST(e.cnt as REAL)  as click
    from (select month, event_type, geo, cnt  from b where event_type=1 ) as s
    left join (select month, event_type, geo, cnt from b where event_type=2 ) as e
        on s.geo = e.geo and s.month = e.month
    group by s.month, s.cnt
    order by s.month, s.cnt
        )
    select month, geo, (click / show) CTR from c
    where CTR is not null
    order by CTR
    LIMIT 5;
         """)

In [32]:
db3

,month,geo,CTR
0,2,2,0.066667
1,2,11,0.083333
2,2,0,0.100000
3,2,4,0.181818
4,2,1,0.200000


## Задание Python

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import time
from django.utils import dateparse
from datetime import timedelta
from datetime import datetime
import sqlite3
import random
path = '/Volumes/Data/ML/mail.ru/'
os.chdir(path)

In [4]:
def got_counter_res_df_speed1(db):
    unique, counts = np.unique(db, return_counts=True); 
    counter = dict(zip(unique, counts))
    res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
    res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)
    res_df['%'] = 0.0;
    for i in (range(len(res_df))): res_df.loc[i, '%'] = int((res_df.loc[i, 'count']*100/(np.sum(res_df['count'])))*1000)/1000
    return res_df

In [2]:
input_filename = 'python_task_dataset.csv'
ds = pd.read_csv(os.path.join(os.getcwd(), input_filename), sep=',', encoding='cp1251')
ds.fillna('', inplace=True)
ds = ds.drop_duplicates().reset_index(drop=True)

In [3]:
ds.head()

,device_id,device_type,datetime,ev_type,adv_type,region_id,age,gender
0,100001,android,2019-03-01 15:58:02,0,multi,2,14,2
1,100001,android,2019-03-01 17:13:53,0,multi,2,14,2
2,100002,android,2019-03-01 23:13:25,0,multi,2,0,0
3,100002,android,2019-03-01 23:25:06,0,multi,2,0,0
4,100003,android,2019-03-01 15:27:53,0,multi,1,0,0


### Задание

В файле находятся данные по показам и кликам рекламы одного рекламодателя в двух регионах.

Описание данных:
* Device_id – уникальный идентификатор девайса отправителя события
* Device_type – тип девайса
* Datetime – время отправки события
* Ev_type – тип события (0 – показ, 1 – клик)
* Adv_type – тип рекламы
* Region_id – регион отправителя события
* Age – возраст отправителя события
* Gender – пол отправителя события (1 – женщины, 2 – мужчины)

Проанализируйте имеющиеся данные и определите:
* реклама для каких групп является более эффективной
* нужно ли рекламодателю устанавливать таргетинги на отдельные группы пользователей (если да, то на какие)
* нужно ли рекламодателю ограничивать время показа рекламы

По результатам анализа сформулируйте рекомендации для рекламодателя (анализ может не ограничиваться перечисленными пунктами).

В качестве решения необходимо прислать код на python и рекомендации (с таблицами/графиками/т.п.).

### Добавлю два новых признака для удобства расчета CTR

In [28]:
ds['click'] = [1 if x == 1 else 0 for x in ds['ev_type']]
ds['show']  = [1 if x == 0 else 0 for x in ds['ev_type']]

### Посмотрю распределение количества уникальных признаков

In [29]:
for c in ds.columns:
    print(c, len(ds[c].unique()))

device_id 187158
device_type 2
datetime 193217
ev_type 2
adv_type 2
region_id 2
age 76
gender 3
click 2
show 2


### Распределение признаков в зависимости от региона

In [6]:
ds1 = ds[ds['region_id'] == 1].reset_index(drop=True)
ds2 = ds[ds['region_id'] == 2].reset_index(drop=True)

for i in [1,3,4,7]:
    res_dfx = got_counter_res_df_speed1(ds1[ds1.columns[i]])
    res_dfy = got_counter_res_df_speed1(ds2[ds2.columns[i]])
    sx = [str(res_dfx.loc[x, 'event'])+'_'+str((res_dfx.loc[x, '%']))+'%' for x in range(len(res_dfx))]
    sy = [str(res_dfy.loc[x, 'event'])+'_'+str((res_dfy.loc[x, '%']))+'%' for x in range(len(res_dfy))]
    print(i, ds1.columns[i], sx, sy)

1 device_type ['android_86.121%', 'ios_13.878%'] ['android_86.17%', 'ios_13.829%']
3 ev_type ['0_98.44%', '1_1.559%'] ['0_98.496%', '1_1.503%']
4 adv_type ['video_80.606%', 'multi_19.393%'] ['video_80.715%', 'multi_19.284%']
7 gender ['1_85.298%', '0_10.896%', '2_3.804%'] ['1_85.72%', '0_9.845%', '2_4.434%']


### Матрица, аналог дерева решений, для расчета CTR в каждой группе уникальных признаков

In [7]:
cols = ['region_id', 'device_type', 'adv_type', 'gender', 'age', 'show', 'click']
res = []
c0 = cols[0]
uniq0 = list(ds[c0].unique())
c1 = cols[1]
uniq1 = list(ds[c1].unique())
c2 = cols[2]
uniq2 = list(ds[c2].unique())
c3 = cols[3]
uniq3 = list(ds[c3].unique())
c4 = cols[4]
uniq4 = list(ds[c4].unique())
    
for i0 in tqdm(range(len(uniq0))):
    ds0 = ds[ds[c0] == uniq0[i0]].reset_index(drop=True)
    
    
    for i1 in (range(len(uniq1))):
        ds1 = ds0[ds0[c1] == uniq1[i1]].reset_index(drop=True)

        for i2 in (range(len(uniq2))):
            ds2 = ds1[ds1[c2] == uniq2[i2]].reset_index(drop=True)
        
            for i3 in (range(len(uniq3))):
                ds3 = ds2[ds2[c3] == uniq3[i3]].reset_index(drop=True)
    
                for i4 in (range(len(uniq4))):
                    ds4 = ds3[ds3[c4] == uniq4[i4]].reset_index(drop=True)
                    
                    if np.sum(ds4[cols[5]]) == 0:
                        CTR = 0
                    else:
                        CTR = np.sum(ds4[cols[6]])/np.sum(ds4[cols[5]])*100.0
                    key = str(i0)+str(i1)+str(i2)+str(i3)+str(i4)
                    res.append([uniq0[i0], uniq1[i1], uniq2[i2], uniq3[i3], uniq4[i4], np.sum(ds4[cols[5]]), np.sum(ds4[cols[6]]), len(ds4), CTR])
    
res = pd.DataFrame(res, columns = cols+['n', 'CTR'])

cols2 = ['device_type', 'adv_type', 'gender', 'age', 'region_id', 'show', 'click', 'n', 'CTR']
res = res[cols2]

100%|██████████| 2/2 [00:01<00:00,  1.35it/s]


In [30]:
res.head(20)

,device_type,adv_type,gender,age,region_id,show,click,n,CTR
0,android,multi,2,14,2,326,1,327,0.306748
1,android,multi,2,0,2,480,1,481,0.208333
2,android,multi,2,38,2,9,0,9,0.000000
3,android,multi,2,16,2,205,1,206,0.487805
4,android,multi,2,21,2,31,0,31,0.000000
5,android,multi,2,46,2,9,0,9,0.000000
6,android,multi,2,75,2,44,0,44,0.000000
7,android,multi,2,33,2,10,0,10,0.000000
8,android,multi,2,36,2,10,0,10,0.000000
9,android,multi,2,12,2,16,0,16,0.000000


### Сделать таблицу более "плоской". Признак региона перенести на остальные признаки

In [9]:
n = int(len(res)/2)
r0 = str(res.iloc[i,4])
r1 = str(res.iloc[i+n,4])
res2 = []        
for i in tqdm(range(n)):
    s0 = list(res.iloc[i,0:4])
    e0 = list(res.iloc[i,5:9])
    e1 = list(res.iloc[i+n,5:9])
    sr = s0+e0+e1
    res2.append(sr)

res2 = pd.DataFrame(res2, columns = cols2[:4]+[x+'_'+r0 for x in cols2[5:9]]+[x+'_'+r1 for x in cols2[5:9]])

100%|██████████| 912/912 [00:00<00:00, 2419.06it/s]


### Добавить признак изменения CTR

In [10]:
for i in tqdm(range(len(res2))):
    res2.loc[i, 'delta'] = res2.loc[i, 'CTR_1'] - res2.loc[i, 'CTR_2']

100%|██████████| 912/912 [00:00<00:00, 4142.94it/s]


### Анализ изменения CTR для каждой уникальной группы в обоих регионах

In [32]:
res2.head(20)

,device_type,adv_type,gender,age,show_2,click_2,n_2,CTR_2,show_1,click_1,n_1,CTR_1,delta
0,android,multi,2,14,326,1,327,0.306748,1155,4,1159,0.346320,0.039572
1,android,multi,2,0,480,1,481,0.208333,1359,4,1363,0.294334,0.086001
2,android,multi,2,38,9,0,9,0.000000,32,0,32,0.000000,0.000000
3,android,multi,2,16,205,1,206,0.487805,432,1,433,0.231481,-0.256323
4,android,multi,2,21,31,0,31,0.000000,34,0,34,0.000000,0.000000
5,android,multi,2,46,9,0,9,0.000000,23,0,23,0.000000,0.000000
6,android,multi,2,75,44,0,44,0.000000,172,0,172,0.000000,0.000000
7,android,multi,2,33,10,0,10,0.000000,33,0,33,0.000000,0.000000
8,android,multi,2,36,10,0,10,0.000000,43,0,43,0.000000,0.000000
9,android,multi,2,12,16,0,16,0.000000,109,0,109,0.000000,0.000000


### Разделить на 2 группы: где было изменение CTR и где не было

In [34]:
# таблица, кто кликал
res3 = res2[res2['delta'] != 0].reset_index(drop=True)
# таблица, кто никак не отреагировал
res4 = res2[res2['delta'] == 0].reset_index(drop=True)

### Разделить на 2 группы: с увеличением и уменьшением CTR

In [14]:
# CTR1 > CTR2
rs1 = res3[res3['delta'] > 0].reset_index(drop=True)
# CTR2 > CTR1
rs2 = res3[res3['delta'] < 0].reset_index(drop=True)

In [39]:
rs1.head()

,device_type,adv_type,gender,age,show_2,click_2,n_2,CTR_2,show_1,click_1,n_1,CTR_1,delta
0,android,multi,2,14,326,1,327,0.306748,1155,4,1159,0.346320,0.039572
1,android,multi,2,0,480,1,481,0.208333,1359,4,1363,0.294334,0.086001
2,android,multi,2,15,234,0,234,0.000000,799,2,801,0.250313,0.250313
3,android,multi,2,8,63,0,63,0.000000,383,3,386,0.783290,0.783290
4,android,multi,2,6,47,0,47,0.000000,284,1,285,0.352113,0.352113


In [40]:
rs2.head()

,device_type,adv_type,gender,age,show_2,click_2,n_2,CTR_2,show_1,click_1,n_1,CTR_1,delta
0,android,multi,2,16,205,1,206,0.487805,432,1,433,0.231481,-0.256323
1,android,multi,2,7,86,1,87,1.162791,452,2,454,0.442478,-0.720313
2,android,multi,2,17,152,1,153,0.657895,303,0,303,0.000000,-0.657895
3,android,multi,2,19,46,1,47,2.173913,120,0,120,0.000000,-2.173913
4,android,multi,2,61,9,1,10,11.111111,11,0,11,0.000000,-11.111111


### Анализ распредения по признакам

In [35]:
for i in [0,1,2]:
    res_dfx = got_counter_res_df_speed1(rs1[rs1.columns[i]])
    res_dfy = got_counter_res_df_speed1(rs2[rs2.columns[i]])
    sx = [str(res_dfx.loc[x, 'event'])+'_'+str((res_dfx.loc[x, '%']))+'%' for x in range(len(res_dfx))]
    sy = [str(res_dfy.loc[x, 'event'])+'_'+str((res_dfy.loc[x, '%']))+'%' for x in range(len(res_dfy))]
    print(i, rs1.columns[i], sx, sy)

0 device_type ['android_61.111%', 'ios_38.888%'] ['android_75.409%', 'ios_24.59%']
1 adv_type ['video_71.111%', 'multi_28.888%'] ['video_77.049%', 'multi_22.95%']
2 gender ['1_86.666%', '2_11.111%', '0_2.222%'] ['1_85.245%', '2_13.114%', '0_1.639%']


### Анализ распредения по возрасту

In [36]:
i = 3
res_dfx = got_counter_res_df_speed1(rs1[rs1.columns[i]])
res_dfy = got_counter_res_df_speed1(rs2[rs2.columns[i]])
sx = [str(res_dfx.loc[x, 'event'])+'_'+str((res_dfx.loc[x, '%']))+'%' for x in range(len(res_dfx))]
sy = [str(res_dfy.loc[x, 'event'])+'_'+str((res_dfy.loc[x, '%']))+'%' for x in range(len(res_dfy))]
print(i, rs1.columns[i], sx, '\n', '\n',sy)

3 age ['53_4.444%', '14_4.444%', '24_3.333%', '15_3.333%', '21_3.333%', '49_2.222%', '48_2.222%', '55_2.222%', '44_2.222%', '57_2.222%', '41_2.222%', '36_2.222%', '34_2.222%', '33_2.222%', '62_2.222%', '29_2.222%', '75_2.222%', '27_2.222%', '6_2.222%', '22_2.222%', '74_2.222%', '8_2.222%', '19_2.222%', '16_2.222%', '73_2.222%', '10_2.222%', '70_1.111%', '54_1.111%', '56_1.111%', '67_1.111%', '58_1.111%', '65_1.111%', '69_1.111%', '68_1.111%', '63_1.111%', '64_1.111%', '52_1.111%', '59_1.111%', '0_1.111%', '50_1.111%', '47_1.111%', '7_1.111%', '9_1.111%', '11_1.111%', '17_1.111%', '20_1.111%', '23_1.111%', '25_1.111%', '30_1.111%', '31_1.111%', '32_1.111%', '37_1.111%', '1_1.111%', '43_1.111%', '45_1.111%', '46_1.111%', '39_1.111%'] 
 
 ['17_4.918%', '0_3.278%', '26_3.278%', '68_3.278%', '61_3.278%', '58_3.278%', '42_3.278%', '40_3.278%', '37_3.278%', '35_3.278%', '31_3.278%', '28_3.278%', '38_3.278%', '19_3.278%', '18_3.278%', '16_3.278%', '13_3.278%', '25_1.639%', '52_1.639%', '70_1.6

In [38]:
# выберем примерно одинаковый процент ТОПов по возрасту
n1 = np.sum(res_dfx.loc[:14, '%'])
n2 = np.sum(res_dfy.loc[:11, '%'])
x = list(res_dfx.loc[:14, 'event'])
y = list(res_dfy.loc[:11, 'event'])
x.sort()
y.sort()

In [18]:
x

[14, 15, 21, 24, 33, 34, 36, 41, 44, 48, 49, 53, 55, 57, 62]

In [19]:
y

[0, 17, 26, 28, 31, 35, 37, 40, 42, 58, 61, 68]

### Сравнение признаков в группах с измениением CTR и без

In [45]:
for i in [0,1,2]:
    res_dfx = got_counter_res_df_speed1(res4[res4.columns[i]])
    res_dfy = got_counter_res_df_speed1(res3[res3.columns[i]])
    sx = [str(res_dfx.loc[x, 'event'])+'_'+str((res_dfx.loc[x, '%']))+'%' for x in range(len(res_dfx))]
    sy = [str(res_dfy.loc[x, 'event'])+'_'+str((res_dfy.loc[x, '%']))+'%' for x in range(len(res_dfy))]
    print(i, rs1.columns[i], sx, sy)

0 device_type ['ios_53.35%', 'android_46.649%'] ['android_66.887%', 'ios_33.112%']
1 adv_type ['multi_54.664%', 'video_45.335%'] ['video_73.509%', 'multi_26.49%']
2 gender ['0_39.553%', '2_37.582%', '1_22.864%'] ['1_86.092%', '2_11.92%', '0_1.986%']


In [46]:
i = 3
res_dfx = got_counter_res_df_speed1(res4[res4.columns[i]])
res_dfy = got_counter_res_df_speed1(res3[res3.columns[i]])
sx = [str(res_dfx.loc[x, 'event'])+'_'+str((res_dfx.loc[x, '%']))+'%' for x in range(len(res_dfx))]
sy = [str(res_dfy.loc[x, 'event'])+'_'+str((res_dfy.loc[x, '%']))+'%' for x in range(len(res_dfy))]
print(i, rs1.columns[i], sx, '\n', '\n', sy)

3 age ['12_1.576%', '2_1.576%', '3_1.576%', '4_1.576%', '5_1.576%', '72_1.576%', '1_1.445%', '64_1.445%', '59_1.445%', '66_1.445%', '67_1.445%', '51_1.445%', '71_1.445%', '11_1.445%', '47_1.445%', '60_1.445%', '56_1.445%', '45_1.445%', '49_1.314%', '43_1.314%', '46_1.314%', '44_1.314%', '42_1.314%', '41_1.314%', '40_1.314%', '50_1.314%', '38_1.314%', '52_1.314%', '54_1.314%', '55_1.314%', '57_1.314%', '39_1.314%', '63_1.314%', '65_1.314%', '69_1.314%', '70_1.314%', '73_1.314%', '74_1.314%', '61_1.314%', '75_1.314%', '27_1.314%', '20_1.314%', '36_1.314%', '22_1.314%', '23_1.314%', '10_1.314%', '25_1.314%', '9_1.314%', '26_1.314%', '18_1.314%', '28_1.314%', '8_1.314%', '7_1.314%', '30_1.314%', '6_1.314%', '32_1.314%', '33_1.314%', '34_1.314%', '35_1.314%', '13_1.314%', '37_1.182%', '68_1.182%', '58_1.182%', '62_1.182%', '21_1.182%', '24_1.182%', '29_1.182%', '48_1.182%', '31_1.182%', '0_1.182%', '17_1.051%', '16_1.051%', '15_1.051%', '19_1.051%', '53_1.051%', '14_0.919%'] 
 
 ['14_3.311%

In [47]:
# выберем примерно одинаковый процент ТОПов по возрасту
n1 = np.sum(res_dfx.loc[:18, '%'])
n2 = np.sum(res_dfy.loc[:11, '%'])
x = list(res_dfx.loc[:14, 'event'])
y = list(res_dfy.loc[:11, 'event'])
x.sort()
y.sort()

In [48]:
# возраст из группы без изменения CTR
x

[1, 2, 3, 4, 5, 11, 12, 47, 51, 59, 64, 66, 67, 71, 72]

In [49]:
# возраст из группы с изменением CTR
y

[0, 14, 15, 16, 17, 19, 24, 29, 31, 37, 48, 53]